In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

In [ ]:
import yaml
import missingno as mn

# Data Loading

In [ ]:
# df = pd.read_csv('/home/geouser05/geo/notebooks/prediction_dts.csv')
df = pd.read_csv('E:/Public/GeoHack/data/02_preprocessed/well_logs.csv')
#df = pd.read_csv('E:/Public/GeoHack/data/07_model_output/prediction_dts_full2.csv')
df = df.iloc[:, 1:].copy()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:

features_base = [
            # 'CALI', 
            # 'DRHO', 
            'DTC', 
            # 'FORCE_2020_LITHOFACIES_CONFIDENCE', 
            #'FORCE_2020_LITHOFACIES_LITHOLOGY',
            'GR', 
            'NPHI', 
            # 'PEF', 
            #'RDEP', 
            'RHOB', 
            #'RMED', 
            # 'ROP',
            # 'RSHA', 
            # 'RXO', 
            # 'SP', 
            # 'RMIC', 
            # 'SGR', 
            # 'ROPA', 
            # 'DCAL',
            # 'MUDWEIGHT'
            ]
features_one_hot_encode = ['FORCE_2020_LITHOFACIES_LITHOLOGY']
features_scale = ['GR', 'NPHI', 'DTC', 'RHOB'] #list(set(features_base) - set(features_one_hot_encode))
target = 'DTS'
n = 10000

In [ ]:
with open('E:/Public/GeoHack/conf/parameters.yml', 'r') as file:
    parameters = yaml.safe_load(file)
raw_filenames = parameters['filename_dts']
def process_filename(x):
    return x.replace('/', '_').replace(' ', '_') + '_logs.las'
filenames = [process_filename(x) for x in raw_filenames]

In [ ]:
# Split well for train and test
test_idx = [3, 22,  5, 33, 35,  0, 20,  5]
train_idx = list(set(range(len(filenames))) - set(test_idx))

# Create train and test dataframes
def get_filename(df, f_list, idx_list):
    f_list = [x for i, x in enumerate(f_list) if i in idx_list]
    return df[df.filename.isin(f_list)].copy()

df_train = get_filename(df, filenames, train_idx)
df_test = get_filename(df, filenames, test_idx)

# Data Preprocessing

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pickle

In [ ]:
## One hot encoding

In [ ]:
if True:
    #nc = OneHotEncoder(sparse = False)
    #enc.fit(df[features_one_hot_encode])

    # with open('E:/Public/GeoHack/data/06_model/onehot_encoder.pkl', 'wb') as file:
    #     pickle.dump(enc, file)

    scaler = StandardScaler()
    scaler.fit(df_train[[target] + features_scale])

    with open('E:/Public/GeoHack/data/06_model/scale.pkl', 'wb') as file:
        pickle.dump(scaler, file)
else:
    with open('/home/geouser05/geo/data/06_model/onehot_encoder.pkl', 'rb') as file:
        enc = pickle.load(file)
    encoded_columns = list(enc.categories_[0])[:-1]

    with open('/home/geouser05/geo/data/06_model/scale.pkl', 'rb') as file:
        scaler = pickle.load(file)
    

In [ ]:
# Transform dataset
df_preprocessed = df.copy()

# df_encode = enc.transform(df[features_one_hot_encode])
# encoded_columns = list(enc.categories_[0])
# df_preprocessed[encoded_columns] = df_encode
# encoded_columns = encoded_columns[:-1]

df_preprocessed[[target] + features_scale] = scaler.transform(df[[target] + features_scale])

In [ ]:
features = features_scale #+ encoded_columns
n_features = len(features) + 1

# EDA

In [ ]:
df_filter = df[df_preprocessed .filename.isin(filenames)].copy()

In [ ]:
mn.matrix(df_filter)

In [ ]:
def generate_slices(df_train, feature, target, filter = True):
    X_train = []
    index_arr = []
    for well_name in df_train.filename.unique():
        df_i = df_train[df_train.filename == well_name].copy()
        df_i = df_i[[target] + feature]

        
        x = df_i.values
        min_idx = x.shape[0] - (x.shape[0] // n) * n
        y = x[min_idx:,:].copy()
        
        idx_list = df_i.index[min_idx:].copy()
        y = y.reshape(-1, n, x.shape[1])

        #target_idx = list(df_i.columns).index(target)
        if filter:
            target_idx = 0
            mask = y[:,:, target_idx]
            mask = np.isnan(mask.astype('float64'))
            mask = mask.sum(axis = 1) < 0.9 * n
            n0 = y.shape[0]
            y = y[mask,:,:]
            n1 = y.shape[0]
            print(f'{well_name:20}: {n0:4} to {n1:4} segments')
        X_train.append(y.copy())
        index_arr.append(idx_list)

    X_train = np.concatenate(X_train, axis = 0)
    index_arr = np.concatenate(index_arr, axis = 0)
    
    return X_train, index_arr, [target] + feature
    

In [ ]:
#df_train[encoded_columns] = enc.transform(df_train[features_one_hot_encode])[:, :-1]
df_train[[target] + features_scale] = scaler.transform(df_train[[target] + features_scale])

df_full = df.copy()
#df_full[encoded_columns] = enc.transform(df_full[features_one_hot_encode])[:, :-1]
df_full[[target] + features_scale] = scaler.transform(df_full[[target] + features_scale])

In [ ]:
X_train, _, _ = generate_slices(df_train, feature = features, target = target)

In [ ]:
X_full, index_arr, columns_names = generate_slices(df_full, feature = features, target = target, filter = False)

In [ ]:
X_train.shape, X_full.shape, index_arr.shape

In [ ]:
X_train = X_train.reshape(-1, 1, n * n_features)
X_full = X_full.reshape(-1, 1, n * n_features)

# GAIN

In [ ]:
def generator_builder(num_fet = 80):
      # 80 * 2 = 162
  generator = keras.models.Sequential([
    keras.layers.Input(shape = num_fet * 2), 
    keras.layers.Dense(32, 'relu'),
    keras.layers.Dense(16, 'relu'),
    keras.layers.Dense(16, 'relu'),
    keras.layers.Dense(16, 'relu'),
    keras.layers.Dense(num_fet, 'linear')
  ])
  return generator

def discriminator_builder(num_fet = 80):
      # 80 * 2 = 162
  discriminator = keras.models.Sequential([
      keras.layers.Input(shape = num_fet * 2),
      keras.layers.Dense(32, 'relu'),
      keras.layers.Dense(16, 'relu'),
      keras.layers.Dense(16, 'relu'),
      keras.layers.Dense(num_fet, 'sigmoid'),
  ])

  return discriminator

In [ ]:
def discriminator_loss(m, m_hat):
    temp = tf.math.multiply(m, tf.math.log(m_hat + 1e-8))
    temp += tf.math.multiply(tf.ones_like(m) - m , tf.math.log(tf.ones_like(m_hat) - m_hat + 1e-8))
    return -tf.reduce_mean(temp)

In [ ]:
def generator_loss(m, m_hat, x, x_hat, alpha = 10):
    loss = -tf.math.multiply((tf.ones_like(m)-m), tf.math.log(m_hat+1e-8))
    loss += alpha * tf.multiply(m, (x - x_hat) ** 2)
    return tf.reduce_mean(loss)

In [ ]:
def hint_smapler(batch_size = 128, num_fet = 80, hint_rate = 0.9):
    A = np.random.uniform(0., 1., size = [batch_size, num_fet])
    B = hint_rate > A
    C = 1.*B
    return C

In [ ]:
# Change Optimizer and learning rate

In [ ]:
generator_optimizer = keras.optimizers.Adam(1e-6)
discriminator_optimizer = keras.optimizers.Adam(1e-6)

In [ ]:
generator = generator_builder(num_fet = n_features * n)
discriminator = discriminator_builder(num_fet = n_features * n)

In [ ]:
@tf.function
def train_step(batch, epoch_num):
    
    m = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), tf.ones_like(batch))
    batch_C = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), batch)
    z = np.random.uniform(0, 0.01, size = batch.shape)
    batch = tf.math.multiply(batch_C, m) + tf.math.multiply((tf.ones_like(m) - m), z)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

        X_temp = tf.cast(generator(tf.concat([batch, m], 1), training = True), tf.float64)
        X_hat = tf.math.multiply(m, batch) + tf.math.multiply((tf.ones_like(m) - m), X_temp)

        H = tf.math.multiply(hint_smapler(1, n * n_features),  m)
        M_hat = tf.cast(discriminator(tf.concat([X_hat, H], 1), training = True), tf.float64)

        disc_loss = discriminator_loss(m, M_hat)
        gen_loss = generator_loss(m, M_hat, batch, X_hat, alpha = 10)

    gen_grad = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grad = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gen_grad, generator.trainable_variables))
    discriminator_optimizer.apply_gradients((zip(disc_grad, discriminator.trainable_variables)))
    return gen_loss, disc_loss

In [ ]:
def train(dataset, EPOCHS, model_filepath = None, n_epochs = 10):
    losses_gen = []
    losses_disc  = []
    for epoch in range(1, EPOCHS+1):
        try:
            print(f'Epoch {epoch} / {EPOCHS}:', end=' ')
            t0 = time.time()
            for batch in dataset:
                gen_loss, disc_loss = train_step(batch, epoch)
                losses_gen.append(gen_loss)
                losses_disc.append(disc_loss)
            t1 = time.time()
            print(f'gen_loss = {gen_loss}, disc_loss = {disc_loss}, time = {t1 - t0}')
            if (model_filepath is not None) and (epoch % n_epochs) == 0:
                generator.save(model_filepath + f'gen_{epoch}.h5')
                discriminator.save(model_filepath + f'disc_{epoch}.h5')
        except KeyboardInterrupt:
            break
    if (model_filepath is not None):
        generator.save(model_filepath + f'gen_{epoch}.h5')
        discriminator.save(model_filepath + f'disc_{epoch}.h5')
    return losses_gen, losses_disc

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices(X_train).shuffle(5)

In [ ]:
gen_loss, disc_loss = train(train_data, EPOCHS = 500) #, model_filepath = '/home/geouser05/geo/data/06_model/', n_epochs = 5)

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(gen_loss, color = 'orange')
plt.title('Generator Loss')
plt.xlabel('Iter')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(disc_loss, color = 'orange')
plt.title('Disciriminator Loss')
plt.xlabel('Iter')
plt.ylabel('Loss')
plt.show()

In [ ]:
generated_data = []
for batch in X_full:
    m = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), tf.ones_like(batch))
    batch_C = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), batch)
    z = np.random.uniform(0, 0.01, size = batch.shape)
    batch = tf.math.multiply(batch_C, m) + tf.math.multiply((tf.ones_like(m) - m), z)
    batch_out = generator(tf.concat([batch, m], 1), training = False)
    generated_data.append(batch_out.numpy().copy())
generated_data = np.concatenate(generated_data, axis = 0).reshape(-1, n_features)
#generated_data = scaler.inverse_transform(generated_data)


In [ ]:
generated_data.shape

In [ ]:
df_generated = pd.DataFrame(generated_data, columns = [str(x) + '_PRED_GAIN' for x in columns_names], index = index_arr)

In [ ]:
df_out = pd.merge(df, df_generated, left_index = True, right_index = True, how = 'left')

In [ ]:
mask = df_out['DTS'].isna()
df_out['DTS_PRED_GAIN_MASK'] = np.nan
df_out['DTS_PRED_GAIN_MASK'][mask] = df_out['DTS_PRED_GAIN'][mask]

In [ ]:
import plotly.express as px

In [ ]:
df_out.filename.unique()

In [ ]:
df_i = df_out[df_out.filename == '35_9-10_S_logs.las']
px.scatter(df_i, x = ['DTS', 'DTS_PRED_GAIN_MASK'], y = 'DEPTH_MD', animation_frame = 'filename')

In [ ]:
df_i.shape

df_out.to_csv('/home/geouser05/geo/data/07_model_output/prediction_dts_full2_gain.csv')

In [ ]:
df_out.shape